In [1]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx

from scipy.spatial import cKDTree
from shapely.geometry import Point

from shapely.geometry import box

In [6]:
# load data
file_path = 'C:/Users/b9066009/Documents/PhD/conferences/2022_GISRUK/gisruk_2022/data/gisruk_2022.gpkg'

od = gpd.read_file(file_path, layer='od_extended')
walk_streets = gpd.read_file(file_path, layer='walk_streets')

In [8]:
# get node geometry
# street network
walk_streets = walk_streets[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

od_nodes = pd.merge(od, nodes_walk, left_on='nodeID', right_on='nodeID')
od_nodes = od_nodes.rename(columns={'geometry_x':'geometry_poi', 'geometry_y':'geometry_node'})

C:\Users\b9066009\AppData\Local\Temp\ipykernel_23412\2935199117.py:10: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [59]:
sample = od_nodes.sample(1)
sample

,nodeID,distance,poiID,groupname,categoryname,classname,geometry_poi,geometry_node
191600,35653,433.354004,6372,Park,Park,Park,POINT (338402.600 397318.430),POINT (338157.035 397514.046)


In [69]:
# get nearest nodes

nodes = walk_streets_pdna.get_node_ids(
    [sample.geometry_node.x, sample.geometry_node.y],
    [sample.geometry_poi.x, sample.geometry_poi.y]
)
nodes

0    27171
1     9874
Name: node_id, dtype: int64

In [67]:
route = walk_streets_pdna.shortest_path(nodes[0],nodes[1])
route

array([27171, 27172, 27166, 27167, 27030, 27029, 26997, 26207, 26203,
       26204, 26206, 26202, 26982, 26985, 26984, 26988, 26989, 27046,
       27050, 27080, 26953, 26950, 26954, 27082, 26907, 26162, 26159,
       26158, 26156, 26155,  5108,  5107,  5608,  4816,  4811,  4812,
        4813, 20538, 20539,  4585,  4980,  4977,  4975,  4974,  4973,
        2887,  2885,  2884,  2264,  2257,  2254,  2255,  2260,  2266,
        2267,  2268,  2272,  2273,  2240,  2241,  2236,  2232,  2233,
        2227,  2226,  2228,  1483,  1482,   736,   734,   735,   737,
        1480,  1474,  1473,  1475,  1477,  1479,  1470,  1466,  1465,
        1464,  1469,  2125,  1459, 13318, 13317, 18391,  3507,  3503,
        3501,  3500,  3504,  3510,  3485,  3484,  3480,  3520,  3518,
        3521, 16282,  4149,  4146,  4144,  4143,  4155,  4154,  4153,
        4326,  4325, 16317,  4162,  4161,  4160,  4164, 17040, 13389,
       13385, 13382, 13366, 13357, 13354, 13353,  6663,  6660,  6664,
        6666,  6661,

In [57]:
ori_x, ori_y = sample.geometry_node.x, sample.geometry_node.y
sample['node_id'] = walk_streets_pdna.get_node_ids(ori_x,ori_y)

In [54]:
ori_x, ori_y = sample.geometry_node.x, sample.geometry_node.y
nodes_ori = walk_streets_pdna.get_node_ids(ori_x, ori_y)

dest_x, dest_y = sample.geometry_poi.x, sample.geometry_poi.y
nodes_dest = walk_streets_pdna.get_node_ids(dest_x, dest_y)

In [55]:
nodes_ori

142808    12967
105604    16280
192572    15052
Name: node_id, dtype: int64

In [ ]:
walk_streets_pdna.get_node_ids()

In [56]:
nodes_dest

142808    12706
105604    16350
192572    21913
Name: node_id, dtype: int64

In [35]:
destinations = list(zip(sample.geometry_poi.x, sample.geometry_poi.y))
origins = list(zip(sample.geometry_node.x, sample.geometry_node.y))

In [45]:
type(destinations)

list

In [43]:
destinations[1]

(344145.66, 388171.7)

In [37]:
nodes = []
for o in origins:
    for d in destinations:
        node = walk_streets_pdna.get_node_ids(
    o[1]
)
nodes

ValueError: setting an array element with a sequence.

In [36]:
routes = []

for o in origins:
    for d in destinations:
        route = walk_streets_pdna.shortest_path(o,d)
        routes.append(route)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
walk_streets_pdna.shortest_path()

In [11]:
origs_nodes = walk_streets_pdna.get_node_ids(postcodes.geometry.x, postcodes.geometry.y).values
dests_nodes = walk_streets_pdna.get_node_ids(destinations.geometry.x, destinations.geometry.y).values

In [21]:
origs_nodes[2]

40490

In [22]:
dests_nodes[5]

2507

In [26]:
%time
routes = []
for o in origs_nodes:
    for d in dests_nodes:
        route = walk_streets_pdna.shortest_path(o,d)
        routes.append(route)

CPU times: total: 0 ns
Wall time: 0 ns


KeyboardInterrupt: 

In [25]:
walk_streets_pdna.shortest_path(origs_nodes[1], dests_nodes[1])

array([41662, 25605, 25591, 25586, 25585, 25587, 27741, 27742, 26387,
       26385, 26388, 25343, 23808, 23803, 23809, 39272, 39274, 39257,
       39258, 39251, 39250, 39249, 32523, 25340, 25335, 25332, 25333,
       25339, 32492, 32489, 32488, 32473, 25330, 25326, 25324, 25315,
       25313, 25314, 25412, 25411, 25410, 25414, 31118, 31123, 25285,
       25284, 25283, 25264, 31119, 25260, 25254, 25261, 25232, 25230,
       25231, 25233, 31144, 31148, 31163, 25384, 25380, 25378, 25379,
       25381, 25389, 27338, 27337, 27341, 27368, 27369, 25189, 25184,
       25185, 25186, 30608, 30605, 30604, 25180, 25174, 25175, 25176,
       30578, 30579, 30504, 30563, 30562, 23327, 23326, 23328, 28299,
       23324, 23321, 23317, 23318, 23312, 23310, 23305, 23303, 23304,
       23307, 32141, 27306, 27305, 27304, 27303, 23295, 23290, 23291,
       23292, 24911, 24912, 24584,  2670,  1968,  2671,  2677,  1977,
         107,   108,   109,  3108,  3109,  3110,   105,   104,    99,
          98,    97,